In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts='140.137.41.81', port=9200)

In [2]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
import operator
import math

In [3]:
#建立ling-林郁芮02text資料庫

from elasticsearch import Elasticsearch
import json

def create_index(es):
    body = dict()
    body['settings'] = get_setting()
    body['mappings'] = get_mappings()
    print(json.dumps(body)) #可以用json.dumps輸出來看格式有沒又包錯
    es.indices.create(index='ling-林郁芮02text', body=body)

def get_setting():
    settings = {
        "index": {
            "number_of_shards": 3,
            "number_of_replicas": 2
        }
    }
    return settings

def get_mappings():
    mappings = {
        "properties": {
            #編號
            "Number": {
                "type": "integer"
            },
            #編號
            "name": {
                "type": "keyword"
            },
            #影片類別
            "video type": {
                "type": "keyword"
            },
            #平均
            "average": {
                "type": "float"
            },
            #一階差
            "first order difference": {
                "type": "float"
            },
            #二階差
            "second order difference": {
                "type": "float"
            },
            #最大心率
            "max heart": {
                "type": "integer"
            },
            #最小心率
            "min heart": {
                "type": "integer"
            },
            #心率變化範圍
            "max to min scope heart": {
                "type": "integer"
            },
            #資訊熵
            "entropy of information": {
                "type": "float"
            },
            #均方根
            "root mean square": {
                "type": "float"
            },
            #標準化均值
            "standardize average": {
                "type": "float"
            }
        }
    }

    return mappings

if __name__ == "__main__":
    es = Elasticsearch(hosts='140.137.41.81', port=9200)
    create_index(es)

{"settings": {"index": {"number_of_shards": 3, "number_of_replicas": 2}}, "mappings": {"properties": {"Number": {"type": "integer"}, "name": {"type": "keyword"}, "video type": {"type": "keyword"}, "average": {"type": "float"}, "first order difference": {"type": "float"}, "second order difference": {"type": "float"}, "max heart": {"type": "integer"}, "min heart": {"type": "integer"}, "max to min scope heart": {"type": "integer"}, "entropy of information": {"type": "float"}, "root mean square": {"type": "float"}, "standardize average": {"type": "float"}}}}


In [4]:
#將ling-林郁芮數據01的2nd進行計算並且將結果傳送到ling-林郁芮02text資料庫

from elasticsearch import Elasticsearch
import json

# 正常(feel=naturl)心率取5個為一組
SamplesStartA = 0
SamplesStopA = 50

#A視頻(feel=happy)心率取5個為一組
SamplesStartB = 0
SamplesStopB = 50

#C視頻(feel=sad)心率取5個為一組
SamplesStartC = 0
SamplesStopC = 50

name="林郁芮"

arra=[0]*5
x=0
arrb=[0]*5
y=0
arrd=[0]*5
z=0

A=5
B=5
D=5

for n in range(0,25): #迴圈取[a1,a11,a21,...],[a2,a22,a33...],[a3,a23,a33...]...共取25組
    arra=[0]*5
    x=0
    arrb=[0]*5
    y=0
    arrd=[0]*5
    z=0
    A=5
    B=5
    D=5
    SamplesStartA+=1
    SamplesStopA+=1
    SamplesStartB+=1
    SamplesStopB+=1
    SamplesStartC+=1
    SamplesStopC+=1
    for m in range(SamplesStartA,SamplesStopA,10):  #迴圈取a1,a11,a21,a31,a41...一組取5個
        def get_queryA():
            queryA = {
            "query": {
                "bool": {
                    "must": [{
                        "term": {
                            "name": "林郁芮"
                        }
                    },
                        {
                        "term": {
                            "feel": "natural"
                        }
                    },
                    {
                        "term": {
                            "time": m
                        }
                    },
                        {
                        "term": {
                            "trial": "2nd"
                        }
                    }]
                }
            }
        }
            return queryA
        if __name__ == "__main__":
            es = Elasticsearch(hosts='140.137.41.81', port=9200)
            queryA = get_queryA()
            resultA = es.search(index='ling-林郁芮數據01', body=queryA)
            resultaa=json.dumps(resultA, ensure_ascii=False)
            resultPulseA=json.loads(resultaa)
            print("A")
            print(resultPulseA['hits']['hits'][0]['_source']['pulse'])
            aa=resultPulseA['hits']['hits'][0]['_source']['pulse']
            arra[x]=aa
            if x==4:
                print(arra)
                x-=4
            else:
                x+=1
    for p in range(SamplesStartB,SamplesStopB,10):  #迴圈取A視頻心率a1,a11,a21,a31,a41...一組取5個
        def get_queryB():
            queryB = {
            "query": {
                "bool": {
                    "must": [{
                        "term": {
                            "name": "林郁芮"
                        }
                    },
                        {
                        "term": {
                            "feel": "happy"
                        }
                    },
                    {
                        "term": {
                            "time": p
                        }
                    },
                        {
                        "term": {
                            "trial": "2nd"
                        }
                    }]
                }
            }
        }
            return queryB
        if __name__ == "__main__":
            es = Elasticsearch(hosts='140.137.41.81', port=9200)
            queryB = get_queryB()
            resultB = es.search(index='ling-林郁芮數據01', body=queryB)
            resultbb=json.dumps(resultB, ensure_ascii=False)
            resultPulseB=json.loads(resultbb)
            print("B")
            print(resultPulseB['hits']['hits'][0]['_source']['pulse'])
            bb=resultPulseB['hits']['hits'][0]['_source']['pulse']
            arrb[y]=bb
            if y==4:
                print(arrb)
                y-=4
            else:
                y+=1
    for q in range(SamplesStartC,SamplesStopC,10):     #迴圈取C視頻心率a1,a11,a21,a31,a41...一組取5個
        def get_queryC():
            queryC = {
            "query": {
                "bool": {
                    "must": [{
                        "term": {
                            "name": "林郁芮"
                        }
                    },
                        {
                        "term": {
                            "feel": "sad"
                        }
                    },
                    {
                        "term": {
                            "time": q
                        }
                    },
                        {
                        "term": {
                            "trial": "2nd"
                        }
                    }]
                }
            }
        }
            return queryC
        if __name__ == "__main__":
            es = Elasticsearch(hosts='140.137.41.81', port=9200)
            queryC = get_queryC()
            resultC = es.search(index='ling-林郁芮數據01', body=queryC)
            resultcc=json.dumps(resultC, ensure_ascii=False)
            resultPulseC=json.loads(resultcc)
            print("C")
            print(resultPulseC['hits']['hits'][0]['_source']['pulse'])
            cc=resultPulseC['hits']['hits'][0]['_source']['pulse']
            arrd[z]=cc
            if z==4:
                print(arrd)
                z-=4
            else:
                z+=1
    #正常心率平均
    ave = 0.0
    for i in range(0,A):
        ave+=arra[i]
    ave = ave/A

    print("A視頻心率特徵 :")

    #平均
    aveB = 0.0
    for i in range(0,B):
        aveB +=arrb[i]
    aveB = aveB/B
    print("平均: ",aveB)

    #DiffB1 一階差的平均值
    diffB1 = 0.0
    meanB1 = 0.0
    #X[n-1]-X[n]的總和
    for i in range(0,(B-1)):
        meanB1+=abs(arrb[i+1]-arrb[i])
    diffB1 = meanB1/(B-1)
    print("一階差: ",diffB1)

    #DiffB2 二階差的平均值
    diffB2 = 0.0
    meanB2 = 0.0
    #X[n+2]-X[n]的總和
    for i in range(0,(B-2)):
        meanB2 += abs(arrb[i+2]-arrb[i])
    diffB2 = meanB2/(B-2)
    print("二階差: ",diffB2)

    #range 心率變化範圍
    scopeB = 0
    scopemaxB = 0
    scopeminB = 110
    for i in range(0,B):
        #求最大心率
        if arrb[i]>scopemaxB:
            scopemaxB = arrb[i]
        if arrb[i]<scopeminB:
            scopeminB = arrb[i]
    scopeB = scopemaxB-scopeminB
    print("最大心率: ",scopemaxB)
    print("最小心率: ",scopeminB)
    print("心率變化範圍: ",scopeB)

    #H(X)資訊熵
    x = 0
    HB = 0
    arrpb = [0.0]*B
    for i in range(0,B):
        x=0 #初值為0
        #心率出現次數
        for j in range(0,B):
            if arrb[i]==arrb[j]:
                x+=1
        #p[i]機率
        arrpb[i] = x*1.0/B
    for k in range(0,B):
        HB += -arrpb[k]*math.log(arrpb[k],2)
    print("資訊熵: ",HB)

    #Radrm 均方根
    radrmB = 0.0   
    meanrB = 0.0
    #相鄰值平方差的總和
    for i in range(0,(B-1)):
        radrmB += pow(arrb[i+1]-arrb[i],2)
    meanrB = radrmB/B
    print("均方根: ",meanrB)

    #標準化均值
    arrbb=[0.0]*B
    aveb = 0.0
    for i in range(0,B):
        arrbb[i] = arrb[i]-ave
        aveb += arrbb[i]
    aveb = aveb/B
    print("標準化均值: ",aveb)
    def load_datas():
        datas = list()
        datas.append(
        {
            #匯入編號
            "Number": int(n),
            #匯入名字
            "name": name,
            #匯入影片類別
            "video type": "A",
            #匯入平均
            "average": float(aveB),
            #匯入一階差
            "first order difference":float(diffB1),
            #匯入二階差
            "second order difference":float(diffB2),
            #匯入最大心率
            "max heart":int(scopemaxB),
            #匯入最小心率
            "min heart":int(scopeminB),
            #匯入心率變化範圍
            "max to min scope heart":int(scopeB),
            #匯入資訊熵
            "entropy of information":float(HB),
            #匯入均方根
            "root mean square":float(meanrB),
            #匯入標準化均值
            "standardize average":float(aveb),
        }
        )
        return datas

    def create_data(es, datas):
        for data in datas:
            es.index(index='ling-林郁芮02text', body=data) 

    if __name__ == "__main__":
        es = Elasticsearch(hosts='140.137.41.81', port=9200)
        datas = load_datas()
        create_data(es, datas)
    print("C視頻心率特徵: ")

    #平均
    aveD = 0.0
    for i in range(0,D):
        aveD +=arrd[i]
    aveD = aveD/D
    print("平均: ",aveD)

    #DiffD1 一階差的平均值
    diffD1 = 0.0
    meanD1 = 0.0
    #X[n+1]-X[n]的總和
    for i in range(0,(D-1)):
        meanD1 += abs(arrd[i+1]-arrd[i])
    diffD1 = meanD1/(D-1)
    print("一階差: ",diffD1)

    #DiffD2 二階差的平均值
    diffD2 = 0.0
    meanD2 = 0.0
    #X[n+2]-X[n]的總和
    for i in range(0,(D-2)):
        meanD2 += abs(arrd[i+2]-arrd[i])
    diffD2 = meanD2/(D-2)
    print("二階差: ",diffD2)

    #range 心率變化範圍
    scopeD = 0
    scopemaxD = 0
    scopeminD = 110
    for i in range(0,D):
        #求最大心率
        if arrd[i]>scopemaxD:
            scopemaxD = arrd[i]
        #求最小心率
        if arrd[i]<scopeminD:
            scopeminD = arrd[i]
    scopeD = scopemaxD-scopeminD
    print("最大心率: ",scopemaxD)
    print("最小心率: ",scopeminD)
    print("心率變化範圍: ",scopeD)

    #H(X)資訊熵
    x = 0
    HD = 0
    arrpd = [0.0]*D
    for i in range(0,D):
        x = 0 #初值為0 
        #心率出現次數
        for j in range(0,D):
            if arrd[i]==arrd[j]:
                x+=1
        arrpd[i] = x*1.0/D
    for k in range(0,D):
        HD += -arrpd[k]*math.log(arrpd[k],2)
    print("資訊熵: ",HD)

    #Radrm 均方根
    radrmD = 0.0
    meanrD = 0.0
    #相鄰值平方差的總和
    for i in range(0,(D-1)):
        radrmD += pow(arrd[i+1]-arrd[i],2)
    meanrD = radrmD/D
    print("均方根: ",meanrD)

    #標準化均值
    arrdd = [0.0]*D
    aved = 0.0
    for i in range(0,D):
        arrdd[i] = arrd[i] -ave
        aved +=arrdd[i]
    aved = aved/D
    print("標準化均值: ",aved)
    
    #匯入資料
    def load_datas():
        datas = list()
        datas.append(
        {
            #匯入編號
            "Number": int(n),
            #匯入名字
            "name": name,
            #匯入影片類別
            "video type": "C",
            #匯入平均
            "average": float(aveD),
            #匯入一階差
            "first order difference":float(diffD1),
            #匯入二階差
            "second order difference":float(diffD2),
            #匯入最大心率
            "max heart":int(scopemaxD),
            #匯入最小心率
            "min heart":int(scopeminD),
            #匯入心率變化範圍
            "max to min scope heart":int(scopeD),
            #匯入資訊熵
            "entropy of information":float(HD),
            #匯入均方根
            "root mean square":float(meanrD),
            #匯入標準化均值
            "standardize average":float(aved),
        }
        )
        return datas

    def create_data(es, datas):
        for data in datas:
            es.index(index='ling-林郁芮02text', body=data)

    if __name__ == "__main__":
        es = Elasticsearch(hosts='140.137.41.81', port=9200)
        datas = load_datas()
        create_data(es, datas)

A
89
A
76
A
92
A
83
A
83
[89, 76, 92, 83, 83]
B
78
B
80
B
80
B
77
B
83
[78, 80, 80, 77, 83]
C
88
C
80
C
78
C
77
C
78
[88, 80, 78, 77, 78]
A視頻心率特徵 :
平均:  79.6
一階差:  2.75
二階差:  2.6666666666666665
最大心率:  83
最小心率:  77
心率變化範圍:  6
資訊熵:  2.450699332842307
均方根:  9.8
標準化均值:  -4.999999999999995
C視頻心率特徵: 
平均:  80.2
一階差:  3.0
二階差:  4.333333333333333
最大心率:  88
最小心率:  77
心率變化範圍:  11
資訊熵:  2.4506993328423072
均方根:  14.0
標準化均值:  -4.399999999999994
A
88
A
82
A
91
A
84
A
82
[88, 82, 91, 84, 82]
B
80
B
75
B
79
B
78
B
84
[80, 75, 79, 78, 84]
C
89
C
79
C
81
C
79
C
76
[89, 79, 81, 79, 76]
A視頻心率特徵 :
平均:  79.2
一階差:  4.0
二階差:  3.0
最大心率:  84
最小心率:  75
心率變化範圍:  9
資訊熵:  2.321928094887362
均方根:  15.6
標準化均值:  -6.2000000000000055
C視頻心率特徵: 
平均:  80.8
一階差:  4.25
二階差:  4.333333333333333
最大心率:  89
最小心率:  76
心率變化範圍:  13
資訊熵:  2.4506993328423072
均方根:  23.4
標準化均值:  -4.600000000000006
A
89
A
83
A
90
A
80
A
83
[89, 83, 90, 80, 83]
B
81
B
73
B
82
B
79
B
77
[81, 73, 82, 79, 77]
C
87
C
80
C
82
C
82
C
77
[87, 80, 82, 82, 77]
A視頻心率

C視頻心率特徵: 
平均:  79.6
一階差:  2.25
二階差:  2.0
最大心率:  82
最小心率:  78
心率變化範圍:  4
資訊熵:  2.4506993328423072
均方根:  5.4
標準化均值:  -4.800000000000006
A
92
A
83
A
83
A
83
A
80
[92, 83, 83, 83, 80]
B
80
B
77
B
83
B
76
B
80
[80, 77, 83, 76, 80]
C
78
C
77
C
78
C
80
C
79
[78, 77, 78, 80, 79]
A視頻心率特徵 :
平均:  79.2
一階差:  5.0
二階差:  2.3333333333333335
最大心率:  83
最小心率:  76
心率變化範圍:  7
資訊熵:  2.4506993328423072
均方根:  22.0
標準化均值:  -5.000000000000003
C視頻心率特徵: 
平均:  78.4
一階差:  1.25
二階差:  1.3333333333333333
最大心率:  80
最小心率:  77
心率變化範圍:  3
資訊熵:  2.450699332842307
均方根:  1.4
標準化均值:  -5.8000000000000025
A
91
A
84
A
82
A
82
A
81
[91, 84, 82, 82, 81]
B
79
B
78
B
84
B
79
B
79
[79, 78, 84, 79, 79]
C
81
C
79
C
76
C
81
C
81
[81, 79, 76, 81, 81]
A視頻心率特徵 :
平均:  79.8
一階差:  3.0
二階差:  3.6666666666666665
最大心率:  84
最小心率:  78
心率變化範圍:  6
資訊熵:  2.2553093074541164
均方根:  12.4
標準化均值:  -4.2
C視頻心率特徵: 
平均:  79.6
一階差:  2.5
二階差:  4.0
最大心率:  81
最小心率:  76
心率變化範圍:  5
資訊熵:  2.2553093074541164
均方根:  7.6
標準化均值:  -4.4
A
90
A
80
A
83
A
81
A
80
[90, 80, 83, 